In [1]:
## Daniel Marten
## GRCh37 Control Preparation

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import random

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Merge and output for Liftover
# list of intergenic non-orf and intergenic-orf dataframes
norf_dfs = []
orf_dfs = []

for gtfidx in range(10):
    # Reading in all GRCh37 ORFs from Nam
    orf_path = f'/Users/marten/ug-gc/bam-gtf/grch37/sample-ORFs/GRCh37.Ens87.dna_rm.chromosome.all.intergenic_gt122.orfs.sample{gtfidx}.bed'
    norf_path = f'/Users/marten/ug-gc/bam-gtf/grch37/sample-nORFs/GRCh37.Ens87.dna_rm.chromosome.all.intergenic_gt122.norf_sample{gtfidx}.bed'
    new_orf = pd.read_csv(orf_path,sep='\t').reset_index()
    new_norf = pd.read_csv(norf_path,sep='\t').reset_index()
    
    # Formatting - naming with _37
    col_renames = {'level_0':'Chr',
               'level_1':'Gene_Start_37',
               'level_2':'Gene_End_37',
               'level_3':'Name',
               '# All coordinates are 0-indexed, coordinates in ID column are end-inclusive, and coordinates in start/end columns are end-exclusive (as is conventional)':'Strand'
              }
    
    new_orf = new_orf.rename(columns=col_renames)
    new_orf = new_orf.set_index('Name').drop('level_4',axis=1)
    new_orf['Status'] = [f'orf_{gtfidx}']*new_orf.shape[0]
    new_orf['Control_Set'] = (gtfidx//2)+1
    #new_orf['annotation'] = ['intergenics']*new_orf.shape[0]
    
    new_norf = new_norf.rename(columns=col_renames)
    new_norf = new_norf.set_index('Name').drop('level_4',axis=1)
    new_norf['Status'] = [f'norf_{gtfidx}']*new_norf.shape[0]
    new_norf['Control_Set'] = (gtfidx//2)+1
    #new_norf['annotation'] = ['intergenics']*new_norf.shape[0]

    orf_dfs.append(new_orf)
    norf_dfs.append(new_norf)

# Create a mega-dataframe with all orfs and non-orfs
mega_37 = pd.concat(orf_dfs + norf_dfs)

# code to turn the DFs as we have them formatted into properly formatted BEDs with the control set in their name
# code copied from GRCh38 work, has old column names that we correct 
def bedify(input_df):
    ret_df = input_df.reset_index()
    newnames = [ret_df.loc[xi,'Name']+ret_df.loc[xi,'Status']+'_control_set_'+str(ret_df.loc[xi,'Control_Set']) for xi in ret_df.index] 
    ret_df['new_index'] = newnames
    ret_df_2 = ret_df[['Chr','Gene_Start_37','Gene_End_37','new_index','Strand']]
    ret_df_2['zeroes'] = 0
    ret_df_2 = ret_df_2[['Chr','Gene_Start_37','Gene_End_37','new_index','zeroes','Strand']].sort_values(by=['Chr','Gene_Start_37','Gene_End_37'])
    ret_df_2.new_index[1]
    return ret_df_2

threesevenbam = bedify(mega_37)

# NOTICE: UNCOMMENT IF YOU NEED TO CREATE A MERGED BAM TO RUN THRU LIFTOVER 
# threesevenbam.to_csv(r'/Users/marten/ug-gc/merged_unrefined_37bam.bam',sep='\t',index=False,header=False)


In [3]:
threesevenbam

,Chr,Gene_Start_37,Gene_End_37,new_index,zeroes,Strand
23269,chr1,42671,42896,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:4250...,0,+
23288,chr1,58253,58619,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:5560...,0,-
20292,chr1,59878,60067,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:5920...,0,-
38957,chr1,105843,105990,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:1057...,0,+
27000,chr1,107280,107418,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:1065...,0,+
...,...,...,...,...,...,...
1999,chrY,59027971,59028151,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,-
19989,chrY,59209645,59209813,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
1985,chrY,59339711,59339870,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
11991,chrY,59353081,59353330,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+


In [4]:
# Take output above and run through UCSC's online liftover tool, with default settings 
# tool: https://genome.ucsc.edu/cgi-bin/hgLiftOver
# and read back in

In [5]:
# read in liftover outputs and parse 
input_bam_liftedover = pd.read_csv(r'/Users/marten/Downloads/hglft_genome_2be94_2c4590.bed',sep='\t',header=None,encoding="utf-8").reset_index()
# Formatting
# Here, naming with _hg38 
col_renames = {0:'Chr',
           1:'Gene_Start_hg38',
           2:'Gene_End_hg38',
           3:'Name',
           4:'Inclusivity',
           5:'Strand'
          }
input_bam_liftedover = input_bam_liftedover.rename(columns=col_renames)
input_bam_liftedover = input_bam_liftedover[['Chr','Gene_Start_hg38','Gene_End_hg38','Name','Inclusivity','Strand']]#.reset_index(by='Name')
input_bam_liftedover.Gene_Start_hg38 = input_bam_liftedover.Gene_Start_hg38# - input_bam_liftedover.Inclusivity
input_bam_liftedover.set_index('Name',inplace=True)
input_bam_liftedover['Control_Set'] = [relevant_str.split('_')[-1] for relevant_str in input_bam_liftedover.index]
input_bam_liftedover['Status'] = [relevant_str.split(':')[-1].split('_')[0][1:] for relevant_str in input_bam_liftedover.index]




In [6]:
# input_bam_liftedover.Gene_Start_hg38 = input_bam_liftedover.Gene_Start_hg38-1
input_bam_liftedover[input_bam_liftedover.Status=='orf'].iloc[::2001]
# show (20000/2001=about 10) ORFs at random, from a mix of Chrs 


,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status
Name,,,,,,,
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:132308-132529:-orf_7_control_set_4,chr1,132308,132530,1,-,4,orf
GRCh37.Ens87.dna_rm.chr10.intergenic_gt122.orf:71432402-71432536:-orf_2_control_set_2,chr10,69672646,69672781,1,-,2,orf
GRCh37.Ens87.dna_rm.chr12.intergenic_gt122.orf:125207652-125207828:+orf_1_control_set_1,chr12,124723106,124723283,1,+,1,orf
GRCh37.Ens87.dna_rm.chr15.intergenic_gt122.orf:80607539-80607859:+orf_3_control_set_2,chr15,80315197,80315518,1,+,2,orf
GRCh37.Ens87.dna_rm.chr19.intergenic_gt122.orf:30248465-30248638:-orf_2_control_set_2,chr19,29757558,29757732,1,-,2,orf
GRCh37.Ens87.dna_rm.chr2.intergenic_gt122.orf:240639420-240639593:+orf_2_control_set_2,chr2,239717726,239717900,1,+,2,orf
GRCh37.Ens87.dna_rm.chr3.intergenic_gt122.orf:189073255-189073395:+orf_6_control_set_4,chr3,189355466,189355607,1,+,4,orf
GRCh37.Ens87.dna_rm.chr5.intergenic_gt122.orf:89146277-89146447:-orf_0_control_set_1,chr5,89850460,89850631,1,-,1,orf
GRCh37.Ens87.dna_rm.chr7.intergenic_gt122.orf:27256960-27257130:+orf_1_control_set_1,chr7,27217341,27217512,1,+,1,orf


In [7]:
# Make a copy of liftedover controls 
control_df = input_bam_liftedover.copy()
# Print ORFs in here 
control_df[control_df.Status=='orf']

,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status
Name,,,,,,,
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:132308-132529:-orf_7_control_set_4,chr1,132308,132530,1,-,4,orf
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:266925-267119:+orf_0_control_set_1,chr1,297174,297369,1,+,1,orf
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:330011-330244:-orf_6_control_set_4,chr1,489091,489325,1,+,4,orf
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:340419-340610:-orf_8_control_set_5,chr1,478725,478917,1,+,5,orf
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:378478-378666:+orf_0_control_set_1,chr1,440669,440858,1,-,1,orf
...,...,...,...,...,...,...,...
GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:59027971-59028150:-orf_0_control_set_1,chrY,56881824,56882004,1,-,1,orf
GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:59209645-59209812:+orf_9_control_set_5,chrY,57063496,57063664,1,+,5,orf
GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:59339711-59339869:+orf_0_control_set_1,chrY,57193560,57193719,1,+,1,orf


In [8]:
# Add annotations for sorting later
control_df['user_length'] = control_df.Gene_End_hg38 - control_df.Gene_Start_hg38
control_df['removal'] = False
control_df['heirarchy'] = 3
control_df.loc[control_df['Status'].str.contains('norf'),'heirarchy'] = 4
control_df

,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status,user_length,removal,heirarchy
Name,,,,,,,,,,
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:42504-43241.norf_segment:42671-42895:+norf_1_control_set_1,chr1,42671,42896,1,+,1,norf,225,False,4
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:55608-59054.norf_segment:58253-58618:-norf_1_control_set_1,chr1,58253,58619,1,-,1,norf,366,False,4
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:59208-60377.norf_segment:59878-60066:-norf_0_control_set_1,chr1,59878,60067,1,-,1,norf,189,False,4
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:105742-106428.norf_segment:105843-105989:+norf_9_control_set_5,chr1,105843,105990,1,+,5,norf,147,False,4
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:106531-107479.norf_segment:107280-107417:+norf_3_control_set_2,chr1,107280,107418,1,+,2,norf,138,False,4
...,...,...,...,...,...,...,...,...,...,...
GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:59027971-59028150:-orf_0_control_set_1,chrY,56881824,56882004,1,-,1,orf,180,False,3
GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:59209645-59209812:+orf_9_control_set_5,chrY,57063496,57063664,1,+,5,orf,168,False,3
GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:59339711-59339869:+orf_0_control_set_1,chrY,57193560,57193719,1,+,1,orf,159,False,3


In [9]:
# CODE TO: 
# by chromosome, in GRCh38 coordinates, remove controls that: 
# are duplicates of other controls
# overlap other controls
# decided by: 
# -> prioritize ORFs over Non-ORFs
# -> prioritize controls which are longer
# -> prioritize controls which start earlier
# -> in the case of duplicates, keep one in earliest control set 

fixed_list = []
dupl_list = []
removed_list = []
gi = 'Gene_Start_hg38'
ge = 'Gene_End_hg38'

total_dups = 0
total_overlaps = 0
by_heirarchy = 0
by_length = 0
punt = 0

for unique_chrom in sorted(control_df.Chr.unique()):
    print(unique_chrom)
    by_chrom_df = control_df[control_df.Chr==unique_chrom]
    
    by_chrom_df = by_chrom_df.sort_values(by=['Chr',gi,ge])
    
    dupls = by_chrom_df[by_chrom_df.duplicated(['Chr','Gene_Start_hg38','Gene_End_hg38'],keep='first')]
    dupl_list.append(dupls)
    
    by_chrom_df_dd = by_chrom_df[~by_chrom_df.duplicated(['Chr','Gene_Start_hg38','Gene_End_hg38'],keep='first')]
    
    total_dups += dupls.shape[0]
    
    for xi,yi in by_chrom_df_dd.iterrows():
        
        index_start = yi[gi]
        index_end = yi[ge]
        
        query_new = by_chrom_df_dd[(by_chrom_df_dd[gi]>(index_start-1))&(by_chrom_df_dd[ge]<(index_end+1000000))]
        
        for xii,yii in query_new.iterrows():
            
            query_start = yii[gi]
            query_end = yii[ge]
            
            case_1 = query_start in range(index_start,index_end+1)
            case_2 = query_end in range(index_start,index_end+1)
            case_3 = index_start in range(int(query_start),int(query_end)+1)
            
            if (xi!=xii):
                if any([case_1,case_2,case_3]):
                    # instance of overlap!
                        total_overlaps += 1
                        if yii['heirarchy'] > yi['heirarchy']:
                            by_chrom_df_dd.loc[xii,'removal'] = True
                            by_heirarchy += 1
                        elif yii['heirarchy'] < yi['heirarchy']:
                            by_chrom_df_dd.loc[xi,'removal'] = True
                            by_heirarchy += 1
                        else:
                            if yii['user_length']>yi['user_length']:
                                by_chrom_df_dd.loc[xi,'removal'] = True
                                by_length += 1
                            elif yii['user_length']<yi['user_length']:
                                by_chrom_df_dd.loc[xii,'removal'] = True
                                by_length += 1
                            else:
                                punt += 1
                                if yii[gi] > yi[gi]:
                                    by_chrom_df_dd.loc[xii,'removal'] = True
                                else:
                                    by_chrom_df_dd.loc[xi,'removal'] = True
     
    removed_list.append(by_chrom_df_dd[by_chrom_df_dd.removal])
    by_chrom_df_dd = by_chrom_df_dd[~by_chrom_df_dd.removal]
    fixed_list.append(by_chrom_df_dd)
    
print('Total duplicates removed (instances): ',total_dups, '(note that first occurence is kept)') # note that the first occurence is kept 
print('Total non-duplicate overlaps observed: ',total_overlaps)
print('Overlaps decided by heirarchy: ',by_heirarchy)
print('Overlaps decided by length: ',by_length)
print('Overlaps where the first occuring one is taken: ',punt)

fixed_df = pd.concat(fixed_list)

fixed_df.shape
    

chr1
chr10
chr11
chr12
chr13
chr14
chr14_GL000009v2_random
chr15
chr15_KI270850v1_alt
chr16
chr17
chr18
chr19
chr2
chr20
chr21
chr22
chr22_KI270879v1_alt
chr3
chr4
chr4_GL000008v2_random
chr5
chr6
chr7
chr7_KI270803v1_alt
chr8
chr8_KI270821v1_alt
chr9
chrUn_KI270742v1
chrX
chrY
Total duplicates removed (instances):  113 (note that first occurence is kept)
Total non-duplicate overlaps observed:  497
Overlaps decided by heirarchy:  250
Overlaps decided by length:  242
Overlaps where the first occuring one is taken:  5


(39354, 10)

In [10]:
# SANITY CHECK: looking at duplicates and making sure they're removed
dupl_list[5] # "random" removed duplicates by chromosome 

,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status,user_length,removal,heirarchy
Name,,,,,,,,,,
GRCh37.Ens87.dna_rm.chr14.intergenic_gt122.orf:65654142-65654270:+orf_7_control_set_4,chr14,65187424,65187553,1,+,4,orf,129,False,3
GRCh37.Ens87.dna_rm.chr14.intergenic_gt122.orf:72268066-72268188:+orf_7_control_set_4,chr14,71801349,71801472,1,+,4,orf,123,False,3
GRCh37.Ens87.dna_rm.chr14.intergenic_gt122.orf:99413699-99413845:+orf_6_control_set_4,chr14,98947362,98947509,1,+,4,orf,147,False,3
GRCh37.Ens87.dna_rm.chr14.intergenic_gt122.orf:104347821-104348036:+orf_7_control_set_4,chr14,103881484,103881700,1,+,4,orf,216,False,3
GRCh37.Ens87.dna_rm.chr14.intergenic_gt122.orf:104712926-104713135:+orf_8_control_set_5,chr14,104246589,104246799,1,+,5,orf,210,False,3
GRCh37.Ens87.dna_rm.chr14.intergenic_gt122.orf:105970271-105970669:+orf_4_control_set_3,chr14,105503934,105504333,1,+,3,orf,399,False,3


In [11]:
# how the duplicate looks in original
control_df[(control_df.Chr=='chr14') & (control_df.Gene_Start_hg38==98947362)] 


,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status,user_length,removal,heirarchy
Name,,,,,,,,,,
GRCh37.Ens87.dna_rm.chr14.intergenic_gt122.orf:99413699-99413845:+orf_4_control_set_3,chr14,98947362,98947509,1,+,3,orf,147,False,3
GRCh37.Ens87.dna_rm.chr14.intergenic_gt122.orf:99413699-99413845:+orf_6_control_set_4,chr14,98947362,98947509,1,+,4,orf,147,False,3


In [12]:
# how the duplicate looks in final , see how there is only one entry 
fixed_df[(fixed_df.Chr=='chr14') & (fixed_df.Gene_Start_hg38==98947362)] 


,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status,user_length,removal,heirarchy
Name,,,,,,,,,,
GRCh37.Ens87.dna_rm.chr14.intergenic_gt122.orf:99413699-99413845:+orf_4_control_set_3,chr14,98947362,98947509,1,+,3,orf,147,False,3


In [13]:
# SANITY CHECK - making sure removed things are actually removed 
removed_list[len(removed_list)//2] # chosen chromosome of removed samples 

,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status,user_length,removal,heirarchy
Name,,,,,,,,,,
GRCh37.Ens87.dna_rm.chr21.intergenic_gt122:14920064-14921545.norf_segment:14921196-14921342:+norf_3_control_set_2,chr21,13548875,13549022,1,+,2,norf,147,True,4
GRCh37.Ens87.dna_rm.chr21.intergenic_gt122:19923431-19924669.norf_segment:19924300-19924443:+norf_1_control_set_1,chr21,18551982,18552126,1,+,1,norf,144,True,4
GRCh37.Ens87.dna_rm.chr21.intergenic_gt122:20137824-20139495.norf_segment:20139219-20139374:+norf_5_control_set_3,chr21,18766901,18767057,1,+,3,norf,156,True,4
GRCh37.Ens87.dna_rm.chr21.intergenic_gt122:21039616-21040413.norf_segment:21039952-21040134:-norf_3_control_set_2,chr21,19667638,19667821,1,-,2,norf,183,True,4
GRCh37.Ens87.dna_rm.chr21.intergenic_gt122:28094960-28095651.norf_segment:28095175-28095432:-norf_7_control_set_4,chr21,26722856,26723114,1,-,4,norf,258,True,4
GRCh37.Ens87.dna_rm.chr21.intergenic_gt122:31462941-31463650.norf_segment:31463307-31463453:-norf_1_control_set_1,chr21,30090989,30091136,1,-,1,norf,147,True,4
GRCh37.Ens87.dna_rm.chr21.intergenic_gt122:32173865-32175274.norf_segment:32174777-32174986:-norf_3_control_set_2,chr21,30802459,30802669,1,-,2,norf,210,True,4
GRCh37.Ens87.dna_rm.chr21.intergenic_gt122:37390053-37391007.norf_segment:37390416-37390718:+norf_5_control_set_3,chr21,36018118,36018421,1,+,3,norf,303,True,4
GRCh37.Ens87.dna_rm.chr21.intergenic_gt122:43031348-43033576.norf_segment:43032333-43032497:+norf_5_control_set_3,chr21,41612173,41612338,1,+,3,norf,165,True,4


In [14]:
# further sanity check at one of these locations to ensure only one entry 
control_df[(control_df.Chr=='chr21') & (control_df.Gene_Start_hg38==26722856)] # sequences removed 


,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status,user_length,removal,heirarchy
Name,,,,,,,,,,
GRCh37.Ens87.dna_rm.chr21.intergenic_gt122:28094960-28095651.norf_segment:28095175-28095432:-norf_7_control_set_4,chr21,26722856,26723114,1,-,4,norf,258,False,4


In [15]:
# checking to make sure a random sequence is actually removed
# REMOVED becuase it overlapped a higher priority sequence! not because it is a duplicate
fixed_df[(fixed_df.Chr=='chr21') & (fixed_df.Gene_Start_hg38==26722856)] 

,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status,user_length,removal,heirarchy
Name,,,,,,,,,,


In [16]:
# Check using the same code: 
gi = 'Gene_Start_hg38'
ge = 'Gene_End_hg38'
    
for chrom_id in sorted(fixed_df.Chr.unique()):

    print(chrom_id)

    query_df = fixed_df[fixed_df['Chr']==chrom_id]

    for xi,yi in query_df.iterrows():
        index_start = yi[gi]
        index_end = yi[ge]

        query_new = query_df[(query_df[gi]>(index_start-1000000))&(query_df[ge]<(index_end+1000000))]

        for xii,yii in query_new.iterrows():
            query_start = yii[gi]
            query_end = yii[ge]

            if xi!=xii:
                if query_start in range(index_start,index_end+1) or query_end in range(index_start,index_end+1):
                    print(xi,yi)
                    print(xii,yii) 
                    print('Exception(DANGER)')



                elif index_start in range(int(query_start),int(query_end)+1):
                    print(xi,yi)
                    print(xii,yii) 
                    print('Exception(DANGER)')
        

chr1
chr10
chr11
chr12
chr13
chr14
chr14_GL000009v2_random
chr15
chr15_KI270850v1_alt
chr16
chr17
chr18
chr19
chr2
chr20
chr21
chr22
chr22_KI270879v1_alt
chr3
chr4
chr4_GL000008v2_random
chr5
chr6
chr7
chr7_KI270803v1_alt
chr8
chr8_KI270821v1_alt
chr9
chrUn_KI270742v1
chrX
chrY


In [17]:
# Internal duplicates taken care of, now time to do the external (AG and UG)
# Remove controls which overlap with known annotated or unannotated genes 

## now reading in the phylo_df 
phylo_df = pd.read_csv(r'gs://ug-wphu/gtex_analysis/victor_2149+Ens89/Hs_Ens89+2149_PS_seq_etc_hg38.txt',sep='\t')
phylo_df.rename(columns={'Name':'Gene_ID'},inplace=True)
phylo_df.rename(columns={'Protein_ID':'Name'},inplace=True)
phylo_df.set_index('Name',inplace=True)
phylo_df.drop('Gene_ID',inplace=True,axis=1)
phylo_df

,PS,Description,Plength,Gap_Gene?,Chr,Gene_Start_hg38,Gene_End_hg38,Strand,CDS_Start_hg38,CDS_End_hg38,Protein_Sequence,CDS_Sequence
Name,,,,,,,,,,,,
Baz_Hs_1,31,ENST00000308604.5_18272_457,52,Not_Gap_Gene,chr2,NaN,NaN,-,111429413,111429572,MTDTENHDSSPSSTSTCCPPITAGMQLKDSLGPGSNCPLWTLRPLH...,ATGACAGACACTGAAAATCACGACTCATCCCCCTCCAGCACCTCTA...
Baz_Hs_10,31,ENST00000411630.2_23991_594,59,Not_Gap_Gene,chr4,NaN,NaN,+,52713673,52713853,MLVATGQCSRCFMFTFSTFSFNCHNSEVDSVRDRLPQDHSAPANSM...,ATGCTGGTGGCAACAGGGCAGTGTAGCAGGTGCTTCATGTTCACCT...
Baz_Hs_103,31,ENST00000499346.2_27384_333,84,Not_Gap_Gene,chr5,NaN,NaN,-,128082767,128083022,MLGAFRSGPQPLPEPRARCVPQPGLLWALTRRRESPLVTPGLNLEE...,ATGCTGGGGGCTTTCCGGTCGGGGCCGCAGCCGCTTCCGGAGCCGC...
Baz_Hs_108,24,ENST00000501177.3_12701_390,84,Not_Gap_Gene,chr16,NaN,NaN,-,54919086,54925136,MLAEIHPKAGLQSLQFIMELLYWLLEGGDSEDKEDATGNVEMKNIQ...,ATGTTGGCTGAAATTCATCCCAAGGCTGGTCTGCAAAGTCTGCAAT...
Baz_Hs_112,25,ENST00000503704.1_24073_293,52,Not_Gap_Gene,chr4,NaN,NaN,-,82900166,82900435,MRSREAGPKLRRIQEPANGSPGAVSETGGYREERLSDAEIMGKLLA...,ATGCGAAGCAGAGAGGCAGGACCAAAATTGAGGCGAATCCAGGAAC...
...,...,...,...,...,...,...,...,...,...,...,...,...
vdp2013_S4_994,31,HF584391,69,Not_Gap_Gene,chrX,NaN,NaN,-,143628143,143628353,MLYTHNTEFNLKRQICFVPQCKTFVSLCFVKQTQENWYTCTSWVLY...,ATGCTTTATACACATAATACTGAATTTAACCTCAAGAGGCAAATCT...
vdp2013_S4_995,31,HF583960,46,Not_Gap_Gene,chr10,NaN,NaN,+,104020518,104021665,MREWLSIRNMRIKCEIFSCSVKPMSANCISCRMKNATCWLSMRLRN,ATGAGAGAATGGCTCAGCATCAGAAACATGAGAATCAAATGCGAGA...
vdp2013_S4_997,31,HF548108,40,Not_Gap_Gene,chr6,NaN,NaN,+,70860637,70860760,MFAYKGSSYHVSNTSNSINPTPKLASNPVGRYCMIKCLII,ATGTTTGCATATAAGGGAAGTAGTTATCATGTTAGTAATACCTCTA...


In [18]:
# Fill in Gene Start if it is undefined, so we can filter from one set of columns
for xi,yi in phylo_df.iterrows():
    if str(yi['Gene_Start_hg38'] == 'nan'):
        phylo_df.loc[xi,'Gene_Start_hg38'] = phylo_df.loc[xi,'CDS_Start_hg38']
        phylo_df.loc[xi,'Gene_End_hg38'] = phylo_df.loc[xi,'CDS_End_hg38']
    if 'ENSP' in xi[:4]:
        # print('annotated')
        phylo_df.loc[xi,'Status'] = 'annotated'
    else:
        phylo_df.loc[xi,'Status'] = 'unannotated'
    phylo_df.loc[xi,'Control_Set'] = 'victorgenes'

In [19]:
# Code to: 
# remove overlapping controls, very similar to code above 
ug_cleaned_dfs = []
ugag_removal_list = []

victor_overlap = 0
ug_overlap = 0
ag_overlap = 0

for unique_chrom in sorted(fixed_df.Chr.unique()):
    print(unique_chrom)
    by_chrom_df = fixed_df[fixed_df.Chr==unique_chrom]
    by_chrom_df_dd = by_chrom_df.sort_values(by=[gi,ge])
    
    phylo_by_chr = phylo_df[phylo_df.Chr==unique_chrom].sort_values(by=[gi,ge])
    
    total_dups += dupls.shape[0]
    
    for xi,yi in by_chrom_df_dd.iterrows():
        
        index_start = yi[gi]
        index_end = yi[ge]
        
        query_new = phylo_by_chr[(phylo_by_chr[gi]>(index_start-1000000))&(phylo_by_chr[ge]<(index_end+1000000))]
        
        for xii,yii in query_new.iterrows():
            
            query_start = yii[gi]
            query_end = yii[ge]
            
            case_1 = query_start in range(index_start,index_end+1)
            case_2 = query_end in range(index_start,index_end+1)
            case_3 = index_start in range(int(query_start),int(query_end)+1)
            
            if (xi!=xii):
                if any([case_1,case_2,case_3]):
                    by_chrom_df_dd.loc[xi,'removal']=True
                    victor_overlap += 1
                    
                    if yii['Status']=='annotated':
                        ag_overlap += 1
                    elif yii['Status']=='unannotated':
                        ug_overlap += 1
                    else:
                        raise Exception('ayylmao')

     
    ugag_removal_list.append(by_chrom_df_dd[by_chrom_df_dd.removal])
    by_chrom_df_dd = by_chrom_df_dd[~by_chrom_df_dd.removal]
    ug_cleaned_dfs.append(by_chrom_df_dd)
    
print('Total ORF/NORFs removed due to overlap with victor genes: ',victor_overlap)
print('For UG: ',ug_overlap)
print('For Ensembl: ',ag_overlap)

df2 = pd.concat(ug_cleaned_dfs)

df2.shape  

chr1
chr10
chr11
chr12
chr13
chr14
chr14_GL000009v2_random
chr15
chr15_KI270850v1_alt
chr16
chr17
chr18
chr19
chr2
chr20
chr21
chr22
chr22_KI270879v1_alt
chr3
chr4
chr4_GL000008v2_random
chr5
chr6
chr7
chr7_KI270803v1_alt
chr8
chr8_KI270821v1_alt
chr9
chrUn_KI270742v1
chrX
chrY
Total ORF/NORFs removed due to overlap with victor genes:  480
For UG:  182
For Ensembl:  298


(38888, 10)

In [20]:
# Checking again, with Exceptions, to ensure we did not miss anything obvious 

for chrom_id in df2.Chr.unique():

    print(chrom_id)

    query_df = df2[df2['Chr']==chrom_id]

    for xi,yi in query_df.iterrows():
        index_start = yi[gi]
        index_end = yi[ge]
        
        p_by_c = phylo_df[phylo_df.Chr==chrom_id]
        query_new = p_by_c[(p_by_c[gi]>(index_start-1000000))&(p_by_c[ge]<(index_end+1000000))]

        for xii,yii in query_new.iterrows():
            query_start = yii[gi]
            query_end = yii[ge]

            if xi!=xii:
                if query_start in range(index_start,index_end+1) or query_end in range(index_start,index_end+1):
                    print(xi,yi)
                    print(xii,yii) 
                    raise Exception('DANGER')

                elif index_start in range(int(query_start),int(query_end)+1):
                    raise Exception('DANGER')
        

chr1
chr10
chr11
chr12
chr13
chr14
chr14_GL000009v2_random
chr15
chr15_KI270850v1_alt
chr16
chr17
chr18
chr19
chr2
chr20
chr21
chr22
chr22_KI270879v1_alt
chr3
chr4
chr4_GL000008v2_random
chr5
chr6
chr7
chr7_KI270803v1_alt
chr8
chr8_KI270821v1_alt
chr9
chrUn_KI270742v1
chrX
chrY


In [21]:
# a sequence that should've been removed 
ugag_removal_list[len(ugag_removal_list)//3].iloc[2]

Chr                   chr17
Gene_Start_hg38    41936883
Gene_End_hg38      41937015
Inclusivity               1
Strand                    -
Control_Set               3
Status                  orf
user_length             132
removal                True
heirarchy                 3
Name: GRCh37.Ens87.dna_rm.chr17.intergenic_gt122.orf:40093136-40093267:-orf_5_control_set_3, dtype: object

In [22]:
# making sure that it's actually removed 
df2[(df2.Chr=='chr17')&(df2.Gene_Start_hg38==41936883)]

,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status,user_length,removal,heirarchy
Name,,,,,,,,,,


In [23]:
# Display the final dataframe to be processed further, and create dictionary for separate dataframes
display(df2)
df_by_set = {}

,Chr,Gene_Start_hg38,Gene_End_hg38,Inclusivity,Strand,Control_Set,Status,user_length,removal,heirarchy
Name,,,,,,,,,,
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:42504-43241.norf_segment:42671-42895:+norf_1_control_set_1,chr1,42671,42896,1,+,1,norf,225,False,4
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:55608-59054.norf_segment:58253-58618:-norf_1_control_set_1,chr1,58253,58619,1,-,1,norf,366,False,4
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:59208-60377.norf_segment:59878-60066:-norf_0_control_set_1,chr1,59878,60067,1,-,1,norf,189,False,4
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:105742-106428.norf_segment:105843-105989:+norf_9_control_set_5,chr1,105843,105990,1,+,5,norf,147,False,4
GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:106531-107479.norf_segment:107280-107417:+norf_3_control_set_2,chr1,107280,107418,1,+,2,norf,138,False,4
...,...,...,...,...,...,...,...,...,...,...
GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:59027971-59028150:-orf_0_control_set_1,chrY,56881824,56882004,1,-,1,orf,180,False,3
GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:59209645-59209812:+orf_9_control_set_5,chrY,57063496,57063664,1,+,5,orf,168,False,3
GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:59339711-59339869:+orf_0_control_set_1,chrY,57193560,57193719,1,+,1,orf,159,False,3


In [24]:
# code to turn the DFs as we have them formatted into properly formatted BEDs with the control set in their name
def bedify_2(input_df):
    #display(input_df)
    ret_df = input_df.reset_index()
    newnames = ret_df.Name#+ret_df.loc[xi,'Status']+'_control_set_'+str(ret_df.loc[xi,'Control_Set']) for xi in ret_df.index] 
    ret_df['new_index'] = newnames
    ret_df_2 = ret_df[['Chr','Gene_Start_hg38','Gene_End_hg38','new_index','Strand']]
    ret_df_2['zeroes'] = 0 # THIS IS ONLY BECUASE WE FIX FOR INCLUSIVITY BEFOREHAND 
    ret_df_2 = ret_df_2[['Chr','Gene_Start_hg38','Gene_End_hg38','new_index','zeroes','Strand']].sort_values(by=['Chr','Gene_Start_hg38','Gene_End_hg38'])
    ret_df_2.new_index[1]
    return ret_df_2

In [25]:
# check for unique chrs - we do INDEED have non-standard contigs
df2.Chr.unique()

array(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14',
       'chr14_GL000009v2_random', 'chr15', 'chr15_KI270850v1_alt',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21',
       'chr22', 'chr22_KI270879v1_alt', 'chr3', 'chr4',
       'chr4_GL000008v2_random', 'chr5', 'chr6', 'chr7',
       'chr7_KI270803v1_alt', 'chr8', 'chr8_KI270821v1_alt', 'chr9',
       'chrUn_KI270742v1', 'chrX', 'chrY'], dtype=object)

In [26]:
# Output separate beds for each type for each control set

for control_set in sorted(df2.Control_Set.unique()):
    # print(control_set)
    df_temp = df2[df2.Control_Set == control_set].sort_values(by=['Chr','Gene_Start_hg38','Gene_End_hg38'])
    df_temp_orf = df_temp[~df_temp.Status.str.contains('norf')]
    df_temp_norf = df_temp[df_temp.Status.str.contains('norf')]
    print(f'math check: {df_temp_orf.shape[0]} + {df_temp_norf.shape[0]} == {df_temp.shape[0]} ? {df_temp_orf.shape[0] + df_temp_norf.shape[0] == df_temp.shape[0]} ')
    df_by_set[str(control_set)] = {'orf':bedify_2(df_temp_orf),'norf':bedify_2(df_temp_norf)}
    bedify_2(df_temp_orf).to_csv(f'orf_{control_set}_v5.bam',sep='\t',index=False,header=False,encoding="utf-8")
    bedify_2(df_temp_norf).to_csv(f'norf_{control_set}_v5.bam',sep='\t',index=False,header=False,encoding="utf-8")
    # it's going to be some work to convert these back to GRCh37 - key the original by the names

math check: 3926 + 3897 == 7823 ? True 
math check: 3908 + 3872 == 7780 ? True 
math check: 3903 + 3876 == 7779 ? True 
math check: 3881 + 3884 == 7765 ? True 
math check: 3863 + 3878 == 7741 ? True 


In [27]:
bedify_2(df_temp_orf)

,Chr,Gene_Start_hg38,Gene_End_hg38,new_index,zeroes,Strand
0,chr1,478725,478917,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:...,0,+
1,chr1,675222,675432,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:...,0,-
2,chr1,2313025,2313796,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:...,0,-
3,chr1,2921177,2921342,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:...,0,+
4,chr1,3019495,3019666,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122.orf:...,0,+
...,...,...,...,...,...,...
3858,chrY,24152772,24152943,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
3859,chrY,25409413,25409626,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
3860,chrY,25953496,25953709,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
3861,chrY,26530763,26530898,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,-


In [28]:
bedify_2(df2)

,Chr,Gene_Start_hg38,Gene_End_hg38,new_index,zeroes,Strand
0,chr1,42671,42896,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:4250...,0,+
1,chr1,58253,58619,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:5560...,0,-
2,chr1,59878,60067,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:5920...,0,-
3,chr1,105843,105990,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:1057...,0,+
4,chr1,107280,107418,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:1065...,0,+
...,...,...,...,...,...,...
38883,chrY,56881824,56882004,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,-
38884,chrY,57063496,57063664,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
38885,chrY,57193560,57193719,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
38886,chrY,57206930,57207179,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+


In [29]:
grch37_final = bedify_2(df2) # FINAL of complete GRCh37->GRCh38 controls 
grch37_final

,Chr,Gene_Start_hg38,Gene_End_hg38,new_index,zeroes,Strand
0,chr1,42671,42896,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:4250...,0,+
1,chr1,58253,58619,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:5560...,0,-
2,chr1,59878,60067,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:5920...,0,-
3,chr1,105843,105990,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:1057...,0,+
4,chr1,107280,107418,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:1065...,0,+
...,...,...,...,...,...,...
38883,chrY,56881824,56882004,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,-
38884,chrY,57063496,57063664,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
38885,chrY,57193560,57193719,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
38886,chrY,57206930,57207179,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+


In [30]:
# Write this out , later to be passed through GTEx and RNASeq calling and assured that expression is as expected
grch37_final.to_csv('grch37liftover_38888.bed',sep='\t',index=False,header=False,encoding="utf-8")


In [31]:
grch37_final

,Chr,Gene_Start_hg38,Gene_End_hg38,new_index,zeroes,Strand
0,chr1,42671,42896,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:4250...,0,+
1,chr1,58253,58619,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:5560...,0,-
2,chr1,59878,60067,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:5920...,0,-
3,chr1,105843,105990,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:1057...,0,+
4,chr1,107280,107418,GRCh37.Ens87.dna_rm.chr1.intergenic_gt122:1065...,0,+
...,...,...,...,...,...,...
38883,chrY,56881824,56882004,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,-
38884,chrY,57063496,57063664,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
38885,chrY,57193560,57193719,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
38886,chrY,57206930,57207179,GRCh37.Ens87.dna_rm.chrY.intergenic_gt122.orf:...,0,+
